In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd D:\StonyBrook\Study\Prob&Stats CSE544\Project

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Prob_stats_proj


In [ ]:
data = pd.read_csv('7.csv')

In [ ]:
## converting date column to datetime data type ##
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data

,Date,IA confirmed cumulative,ID confirmed cumulative,IA deaths cumulative,ID deaths cumulative,IA confirmed,ID confirmed,IA deaths,ID deaths
0,2020-01-22,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
433,2021-03-30,349742,179429,5726,1956,141,0,7,0
434,2021-03-31,350840,180536,5744,1962,1098,1107,18,6
435,2021-01-04,351651,180897,5744,1963,811,361,0,1
436,2021-02-04,352262,181181,5752,1966,611,284,8,3


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     438 non-null    datetime64[ns]
 1   IA confirmed cumulative  438 non-null    int64         
 2   ID confirmed cumulative  438 non-null    int64         
 3   IA deaths cumulative     438 non-null    int64         
 4   ID deaths cumulative     438 non-null    int64         
 5   IA confirmed             438 non-null    int64         
 6   ID confirmed             438 non-null    int64         
 7   IA deaths                438 non-null    int64         
 8   ID deaths                438 non-null    int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 30.9 KB


In [ ]:
###### Feb and March data #######
feb_start_date , feb_end_date = '2021-02-01', '2021-02-28'
march_start_date, march_end_date = '2021-03-01', '2021-03-31'
condition = (data['Date'] >= feb_start_date) & (data['Date'] <= feb_end_date)
feb_data = data.loc[condition]

condition = (data['Date'] >= march_start_date) & (data['Date'] <= march_end_date)
march_data = data.loc[condition]

print(len(march_data))

23


In [ ]:
########################### Tests for IA confirmed cases handle ###################################

In [ ]:
handle = 'IA confirmed'
mean_feb = np.mean(feb_data[handle])

## Using mean_feb as guess for mean_march ##

# Wald's test # 
# w = (theta_hat - theta_0)/ se_hat(theta_hat)
# theta_hat is estimator of theta

# Null Hypothesis : mean_march = mean_feb
# Alternate Hypothesis : mean_march != mean_feb

# Assuming the distribution of march data to be poisson. MLE_mean = Sample_mean

sample_mean_march = np.mean(march_data[handle])

standard_error_estimate = np.sqrt(sample_mean_march/len(march_data))

walds_statistic = np.abs((sample_mean_march - mean_feb)/standard_error_estimate)

print('MLE for March data ',sample_mean_march )
print('Guess mean ', mean_feb)
print('Standard Error ', standard_error_estimate)
print('Walds Statistic', walds_statistic )

# Accept Null Hypothesis(less than 1.96)

MLE for March data  539.695652173913
Guess mean  547.0
Standard Error  4.844071464727531
Walds Statistic 1.5078943156132505


In [ ]:
## Z-test ##

# z_statistic = (sample_mean - guess)/ root(true_variance/n)
# true variance = corrected sample standard deviation

sample_mean_full_data = np.mean(data[handle])

true_variance = np.sum(np.square(data[handle] - sample_mean_full_data))/(len(data)-1)

z_statistic = np.abs((sample_mean_march - mean_feb)/(np.sqrt(true_variance)/np.sqrt(len(march_data))))

print(' True Variance ', true_variance)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' z_statistic ', z_statistic)


# Accept Null Hypothesis(less than 1.96)

 True Variance  1040931.0330971861
 Sample Mean  539.695652173913
 Guess  547.0
 z_statistic  0.03433478921672646


In [ ]:
## t-test ##

# t_statistic = (sample_mean - guess)/ corrected_sample_standard_deviation/root(n)
corrected_sample_SD = np.sqrt(np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1))

t_statistic = np.abs((sample_mean_march - mean_feb)/(corrected_sample_SD/np.sqrt(len(march_data))))

print(' Corrected Sample Standard Deviation', corrected_sample_SD)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' t_statistic ', t_statistic)

# degrees of freedom : 28 + 31 -2 = 57
# threshold = 2.002465

# Accept Null Hypothesis

 Corrected Sample Standard Deviation 346.5688223377873
 Sample Mean  539.695652173913
 Guess  547.0
 t_statistic  0.10107782149960952


In [ ]:
## Walds - 2 sample test ##

# delta = mean_march - mean_feb 

# w_stat = delta_hat/ SE_hat(delta_hat)

# Assumption : Data is poisson distributed

# Null Hypothesis : delta = 0
# Alternate Hypothesis : delta != 0 


sample_mean_march = np.mean(march_data[handle])
sample_mean_feb = np.mean(feb_data[handle])

delta_hat = np.abs(sample_mean_march - sample_mean_feb)

SE_hat = np.sqrt(sample_mean_march/len(march_data) + sample_mean_feb/len(feb_data))

w_stat = np.abs(delta_hat/SE_hat)

print(' Standard Error Estimate', SE_hat)
print(' Delta Estimate ',delta_hat )
print(' w_stat ', w_stat)

# Accept Null Hypothesis

 Standard Error Estimate 7.128466059075229
 Delta Estimate  7.304347826086996
 w_stat  1.024673157668732


In [ ]:
## 2 sample unpaired t-test ##

# D_bar = X_bar - Y_bar
# t_stat = D_bar/root(corrected_var_x/n + corrected_var_y/m)

corrected_variance_march = np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1)
corrected_variance_feb =  np.sum(np.square(feb_data[handle] - sample_mean_feb))/(len(feb_data)-1)

D_bar = np.abs(sample_mean_march - sample_mean_feb)

t_stat = D_bar/np.sqrt(corrected_variance_march/len(march_data) + corrected_variance_feb/len(feb_data))

print(' Corrected Variance March', corrected_variance_march)
print(' Corrected Variance Feb', corrected_variance_feb)
print(' D bar',D_bar )
print(' t_statistic ', t_stat)

# threshold = 2.002465

# Accept Null Hypothesis

 Corrected Variance March 120109.94861660078
 Corrected Variance Feb 27054.315789473683
 D bar 7.304347826086996
 t_statistic  0.09008184730273948


In [ ]:
########################### Tests for IA death cases handle ###################################

In [ ]:
handle = 'IA deaths'
mean_feb = np.mean(feb_data[handle])

## Using mean_feb as guess for mean_march ##

# Wald's test # 
# w = (theta_hat - theta_0)/ se_hat(theta_hat)
# theta_hat is estimator of theta

# Null Hypothesis : mean_march = mean_feb
# Alternate Hypothesis : mean_march != mean_feb

# Assuming the distribution of march data to be poisson. MLE_mean = Sample_mean

sample_mean_march = np.mean(march_data[handle])

standard_error_estimate = np.sqrt(sample_mean_march/len(march_data))

walds_statistic = np.abs((sample_mean_march - mean_feb)/standard_error_estimate)

print('Walds test')
print('MLE for March data ',sample_mean_march )
print('Guess mean ', mean_feb)
print('Standard Error ', standard_error_estimate)
print('Walds Statistic', walds_statistic )

# Reject Null Hypothesis(greater than 1.96)
print('########################################')


## Z-test ##

# z_statistic = (sample_mean - guess)/ root(true_variance/n)
# true variance = corrected sample standard deviation

sample_mean_full_data = np.mean(data[handle])

true_variance = np.sum(np.square(data[handle] - sample_mean_full_data))/(len(data)-1)

z_statistic = np.abs((sample_mean_march - mean_feb)/(np.sqrt(true_variance)/np.sqrt(len(march_data))))

print('Z test')
print(' True Variance ', true_variance)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' z_statistic ', z_statistic)


# Reject Null Hypothesis(greater than 1.96)

print('########################################')


## t-test ##

# t_statistic = (sample_mean - guess)/ corrected_sample_standard_deviation/root(n)
corrected_sample_SD = np.sqrt(np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1))

t_statistic = np.abs((sample_mean_march - mean_feb)/(corrected_sample_SD/np.sqrt(len(march_data))))

print('T-test')
print(' Corrected Sample Standard Deviation', corrected_sample_SD)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' t_statistic ', t_statistic)

# degrees of freedom : 28 + 31 -2 = 57
# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

## Walds - 2 sample test ##

# delta = mean_march - mean_feb 

# w_stat = delta_hat/ SE_hat(delta_hat)

# Assumption : Data is poisson distributed

# Null Hypothesis : delta = 0
# Alternate Hypothesis : delta != 0 


sample_mean_march = np.mean(march_data[handle])
sample_mean_feb = np.mean(feb_data[handle])

delta_hat = np.abs(sample_mean_march - sample_mean_feb)

SE_hat = np.sqrt(sample_mean_march/len(march_data) + sample_mean_feb/len(feb_data))

w_stat = np.abs(delta_hat/SE_hat)

print('Walds - 2 sample test')
print(' Standard Error Estimate', SE_hat)
print(' Delta Estimate ',delta_hat )
print(' w_stat ', w_stat)

# Reject Null Hypothesis

print('########################################')

## 2 sample unpaired t-test ##

# D_bar = X_bar - Y_bar
# t_stat = D_bar/root(corrected_var_x/n + corrected_var_y/m)

corrected_variance_march = np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1)
corrected_variance_feb =  np.sum(np.square(feb_data[handle] - sample_mean_feb))/(len(feb_data)-1)

D_bar = np.abs(sample_mean_march - sample_mean_feb)

t_stat = D_bar/np.sqrt(corrected_variance_march/len(march_data) + corrected_variance_feb/len(feb_data))

print('2 sample unpaired t-test ')
print(' Corrected Variance March', corrected_variance_march)
print(' Corrected Variance Feb', corrected_variance_feb)
print(' D bar',D_bar )
print(' t_statistic ', t_stat)

# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

Walds test
MLE for March data  7.565217391304348
Guess mean  17.15
Standard Error  0.5735176503596922
Walds Statistic 16.71227137069692
########################################
Z test
 True Variance  995.7540150256522
 Sample Mean  7.565217391304348
 Guess  17.15
 z_statistic  1.4567001067158405
########################################
T-test
 Corrected Sample Standard Deviation 12.619703522509846
 Sample Mean  7.565217391304348
 Guess  17.15
 t_statistic  3.642478802841167
########################################
Walds - 2 sample test
 Standard Error Estimate 1.0892302306097192
 Delta Estimate  9.584782608695651
 w_stat  8.799592904550924
########################################
2 sample unpaired t-test 
 Corrected Variance March 159.25691699604744
 Corrected Variance Feb 209.50263157894736
 D bar 9.584782608695651
 t_statistic  2.297818982516213
########################################


In [ ]:
########################### Tests for ID confirmed cases handle ###################################

In [ ]:
handle = 'ID confirmed'
mean_feb = np.mean(feb_data[handle])

## Using mean_feb as guess for mean_march ##

# Wald's test # 
# w = (theta_hat - theta_0)/ se_hat(theta_hat)
# theta_hat is estimator of theta

# Null Hypothesis : mean_march = mean_feb
# Alternate Hypothesis : mean_march != mean_feb

# Assuming the distribution of march data to be poisson. MLE_mean = Sample_mean

sample_mean_march = np.mean(march_data[handle])

standard_error_estimate = np.sqrt(sample_mean_march/len(march_data))

walds_statistic = np.abs((sample_mean_march - mean_feb)/standard_error_estimate)

print('Walds test')
print('MLE for March data ',sample_mean_march )
print('Guess mean ', mean_feb)
print('Standard Error ', standard_error_estimate)
print('Walds Statistic', walds_statistic )

# Reject Null Hypothesis(greater than 1.96)
print('########################################')


## Z-test ##

# z_statistic = (sample_mean - guess)/ root(true_variance/n)
# true variance = corrected sample standard deviation

sample_mean_full_data = np.mean(data[handle])

true_variance = np.sum(np.square(data[handle] - sample_mean_full_data))/(len(data)-1)

z_statistic = np.abs((sample_mean_march - mean_feb)/(np.sqrt(true_variance)/np.sqrt(len(march_data))))

print('Z test')
print(' True Variance ', true_variance)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' z_statistic ', z_statistic)


# Reject Null Hypothesis(greater than 1.96)

print('########################################')


## t-test ##

# t_statistic = (sample_mean - guess)/ corrected_sample_standard_deviation/root(n)
corrected_sample_SD = np.sqrt(np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1))

t_statistic = np.abs((sample_mean_march - mean_feb)/(corrected_sample_SD/np.sqrt(len(march_data))))

print('T-test')
print(' Corrected Sample Standard Deviation', corrected_sample_SD)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' t_statistic ', t_statistic)

# degrees of freedom : 28 + 31 -2 = 57
# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

## Walds - 2 sample test ##

# delta = mean_march - mean_feb 

# w_stat = delta_hat/ SE_hat(delta_hat)

# Assumption : Data is poisson distributed

# Null Hypothesis : delta = 0
# Alternate Hypothesis : delta != 0 


sample_mean_march = np.mean(march_data[handle])
sample_mean_feb = np.mean(feb_data[handle])

delta_hat = np.abs(sample_mean_march - sample_mean_feb)

SE_hat = np.sqrt(sample_mean_march/len(march_data) + sample_mean_feb/len(feb_data))

w_stat = np.abs(delta_hat/SE_hat)

print('Walds - 2 sample test')
print(' Standard Error Estimate', SE_hat)
print(' Delta Estimate ',delta_hat )
print(' w_stat ', w_stat)

# Reject Null Hypothesis

print('########################################')

## 2 sample unpaired t-test ##

# D_bar = X_bar - Y_bar
# t_stat = D_bar/root(corrected_var_x/n + corrected_var_y/m)

corrected_variance_march = np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1)
corrected_variance_feb =  np.sum(np.square(feb_data[handle] - sample_mean_feb))/(len(feb_data)-1)

D_bar = np.abs(sample_mean_march - sample_mean_feb)

t_stat = D_bar/np.sqrt(corrected_variance_march/len(march_data) + corrected_variance_feb/len(feb_data))

print('2 sample unpaired t-test ')
print(' Corrected Variance March', corrected_variance_march)
print(' Corrected Variance Feb', corrected_variance_feb)
print(' D bar',D_bar )
print(' t_statistic ', t_stat)

# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

Walds test
MLE for March data  298.0869565217391
Guess mean  333.5
Standard Error  3.6000420077364264
Walds Statistic 9.836841737446083
########################################
Z test
 True Variance  309319.95639112673
 Sample Mean  298.0869565217391
 Guess  333.5
 z_statistic  0.30536777204611804
########################################
T-test
 Corrected Sample Standard Deviation 262.57620764672185
 Sample Mean  298.0869565217391
 Guess  333.5
 t_statistic  0.6468026626311423
########################################
Walds - 2 sample test
 Standard Error Estimate 5.443831597089216
 Delta Estimate  35.413043478260875
 w_stat  6.50516880375139
########################################
2 sample unpaired t-test 
 Corrected Variance March 68946.26482213438
 Corrected Variance Feb 95833.0
 D bar 35.413043478260875
 t_statistic  0.40124871600151857
########################################


In [ ]:
########################### Tests for ID death cases handle ###################################

In [ ]:
handle = 'ID deaths'
mean_feb = np.mean(feb_data[handle])

## Using mean_feb as guess for mean_march ##

# Wald's test # 
# w = (theta_hat - theta_0)/ se_hat(theta_hat)
# theta_hat is estimator of theta

# Null Hypothesis : mean_march = mean_feb
# Alternate Hypothesis : mean_march != mean_feb

# Assuming the distribution of march data to be poisson. MLE_mean = Sample_mean

sample_mean_march = np.mean(march_data[handle])

standard_error_estimate = np.sqrt(sample_mean_march/len(march_data))

walds_statistic = np.abs((sample_mean_march - mean_feb)/standard_error_estimate)

print('Walds test')
print('MLE for March data ',sample_mean_march )
print('Guess mean ', mean_feb)
print('Standard Error ', standard_error_estimate)
print('Walds Statistic', walds_statistic )

# Reject Null Hypothesis(greater than 1.96)
print('########################################')


## Z-test ##

# z_statistic = (sample_mean - guess)/ root(true_variance/n)
# true variance = corrected sample standard deviation

sample_mean_full_data = np.mean(data[handle])
true_variance = np.sum(np.square(data[handle] - sample_mean_full_data))/(len(data)-1)

z_statistic = np.abs((sample_mean_march - mean_feb)/(np.sqrt(true_variance)/np.sqrt(len(march_data))))

print('Z test')
print(' True Variance ', true_variance)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' z_statistic ', z_statistic)


# Reject Null Hypothesis(greater than 1.96)

print('########################################')


## t-test ##

# t_statistic = (sample_mean - guess)/ corrected_sample_standard_deviation/root(n)
corrected_sample_SD = np.sqrt(np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1))

t_statistic = np.abs((sample_mean_march - mean_feb)/(corrected_sample_SD/np.sqrt(len(march_data))))

print('T-test')
print(' Corrected Sample Standard Deviation', corrected_sample_SD)
print(' Sample Mean ',sample_mean_march )
print(' Guess ', mean_feb)
print(' t_statistic ', t_statistic)

# degrees of freedom : 28 + 31 -2 = 57
# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

## Walds - 2 sample test ##

# delta = mean_march - mean_feb 

# w_stat = delta_hat/ SE_hat(delta_hat)

# Assumption : Data is poisson distributed

# Null Hypothesis : delta = 0
# Alternate Hypothesis : delta != 0 


sample_mean_march = np.mean(march_data[handle])
sample_mean_feb = np.mean(feb_data[handle])

delta_hat = np.abs(sample_mean_march - sample_mean_feb)

SE_hat = np.sqrt(sample_mean_march/len(march_data) + sample_mean_feb/len(feb_data))

w_stat = np.abs(delta_hat/SE_hat)

print('Walds - 2 sample test')
print(' Standard Error Estimate', SE_hat)
print(' Delta Estimate ',delta_hat )
print(' w_stat ', w_stat)

# Reject Null Hypothesis

print('########################################')

## 2 sample unpaired t-test ##

# D_bar = X_bar - Y_bar
# t_stat = D_bar/root(corrected_var_x/n + corrected_var_y/m)

corrected_variance_march = np.sum(np.square(march_data[handle] - sample_mean_march))/(len(march_data)-1)
corrected_variance_feb =  np.sum(np.square(feb_data[handle] - sample_mean_feb))/(len(feb_data)-1)

D_bar = np.abs(sample_mean_march - sample_mean_feb)

t_stat = D_bar/np.sqrt(corrected_variance_march/len(march_data) + corrected_variance_feb/len(feb_data))

print('2 sample unpaired t-test ')
print(' Corrected Variance March', corrected_variance_march)
print(' Corrected Variance Feb', corrected_variance_feb)
print(' D bar',D_bar )
print(' t_statistic ', t_stat)

# threshold = 2.002465

# Reject Null Hypothesis

print('########################################')

Walds test
MLE for March data  2.739130434782609
Guess mean  4.95
Standard Error  0.3450979970953814
Walds Statistic 6.406497817506401
########################################
Z test
 True Variance  57.36256961641746
 Sample Mean  2.739130434782609
 Guess  4.95
 z_statistic  1.3999506946006743
########################################
T-test
 Corrected Sample Standard Deviation 3.3739293426931987
 Sample Mean  2.739130434782609
 Guess  4.95
 t_statistic  3.1426141089067876
########################################
Walds - 2 sample test
 Standard Error Estimate 0.6054689319851547
 Delta Estimate  2.2108695652173913
 w_stat  3.651499603734579
########################################
2 sample unpaired t-test 
 Corrected Variance March 11.38339920948616
 Corrected Variance Feb 28.99736842105264
 D bar 2.2108695652173913
 t_statistic  1.585352274044677
########################################
